# Training a Contrail Classifier
The goal of this project is to train a machine learning model that can accurately classify images of the sky as containing contrails.
To build the model, we have obtained cloud data from four sources:
1. [Cirrus Cumulus Stratus Nimbus (CCSN) Database](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/CADDPD)
2. [Singapore Data Swimcat](https://ieeexplore.ieee.org/abstract/document/7350833)
4. [CLASA](https://github.com/CLASA/Contrail-Machine-Vision), [Official Website](https://clasa.github.io/) a proposed solution to the NASA Clouds vs Contrails challenge.

Potential applications are noted below:

Potential Applications
* Climate Studies: Contrails can have a significant impact on the Earth's atmosphere and climate. They can reflect sunlight back into space, contributing to global cooling, but they can also trap heat within the Earth's atmosphere, contributing to global warming. Therefore, a machine learning model trained to detect and monitor contrails could provide important data for climate researchers.

* Air Traffic Control: A model trained to identify contrails could be useful in tracking aircraft routes and densities, particularly in areas with less developed radar infrastructure.

* Aerospace and Defense: This model could be used for aerospace and defense purposes. For instance, detecting contrails could help in tracking and identifying stealth, unauthorized, or unrecognized flights, which can be important in maintaining airspace security.

# Preprocessing Script
Preprocessing is a crucial step in the machine learning pipeline because the quality and quantity of the data that you feed into your model will directly determine how well it can learn. Here are some reasons why we preprocess image data:
1. Labeling: These images are not all labeled, and images from different datasets. The purpose of labeling is to homogenize the data so that each image is labeled in the same manner.

2. Image Resizing: In real-world scenarios, images can come in different sizes and aspect ratios. However, many computer vision models (like Convolutional Neural Networks) require images to be of a uniform size. Therefore, images often need to be resized to fit the requirements of the model.

3. Normalization: Image pixel intensities can range from 0 to 255. Normalizing these pixel intensities to a smaller range, often between 0 and 1 or -1 and 1, can help the model learn more effectively. This is because smaller, centered values are easier for the model's weight initialization and optimization process.

4. Data Augmentation: Image datasets can be augmented by applying random transformations like rotation, scaling, translation, flip etc. This can help increase the amount of training data and make the model more robust to variations in the input data that it hasn't seen before.

5. Dealing with Color Channels: Some models might require grayscale images, while others might require color images. Depending on the model, you might need to convert images from color to grayscale, or vice versa.

6. Feature Extraction: In some cases, it might be beneficial to manually extract features from the images, such as edges, corners, and other local features. These can be used as inputs to the machine learning model.

7. Dimensionality Reduction: Images are high-dimensional data, and it may be beneficial to reduce their dimensionality. This can be done through techniques like Principal Component Analysis (PCA) or autoencoders, which can make the model more efficient without losing too much information.

These preprocessing steps help to make the image data more suitable for computer vision models and can lead to better performance.

## Loading the Data and Labeling

In [ ]:
import os
from PIL import Image

In [ ]:
def image_loader(image_dir, dictionary, filetype):
    # List to hold all image data
    images = []
    # List to hold image classifications (1,0)
    classes = []
    # Iterate over each image in the directory
    for folder in os.listdir(image_dir):
        # Only open files with .jpg extension (or add your extension)
        for filename in os.listdir(image_dir+"/"+folder):
            if filename.endswith(filetype):
                # Open each image file
                img = Image.open(os.path.join(image_dir, filename))
                # Append the image data to your list
                images.append(img)
                classes.append(dictionary[folder])
    # Now the 'images' list contains all the images im the image_dir as PIL Image objects, with the labels in the 'classes' list
    return(images, classes)

In [ ]:
image_dir = "data/CCSN_v2"
# Dictionary to translate scientific names into contrail labels
ccsn_dictionary = {
    'Ct':1,
    'Ac':0, 'Sc':0, 'Ns':0, 'Cu':0, 'Ci':0, 'Cc':0, 'Cb':0, 'As':0,  'Cs':0, 'St':0
}
images_ccsn, classes_ccsn = image_loader(image_dir, ccsn_dictionary, ".jpg")

In [ ]:
image_dir = "data/CLASA"
# Dictionary to translate names into contrail labels
ccsn_dictionary = {
    'Contrail':1,
    'Cirrus':0
}
images_clasa, classes_clasa = image_loader(image_dir, ccsn_dictionary, ".jpg")

In [ ]:
image_dir = "data/Singapore Data Swimcat"
# Dictionary to SWIMCAT labels into contrail labels
ccsn_dictionary = {
    'A-sky':0,
    'B-pattern':0,
    'C-thick-dark':0,
    'D-thick-white':0,
    'E-veil':0
}
images_singapore, classes_singapore = image_loader(image_dir, ccsn_dictionary, ".jpg")

In [ ]:
# Merge all the folders into two lists: one containing images, and the other has the labels
images_all = images_ccsn.append(images_clasa)
images_all = images_all.append(images_singapore)

classes_all = classes_ccsn.append(classes_clasa)
classes_all = classes_all.append(classes_singapore)

## Train test split

In [ ]:
from sklearn.model_selection import train_test_split

# Assume X is your array of features and y are the labels
X_train, X_test, y_train, y_test = train_test_split(images_all, classes_all, test_size=0.2, random_state=42)

## Normalization

## Dealing with Color Channels

## Feature Extraction

## Dimensionality Reduction

## Image Resizing

In [1]:
from keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
random_datagen = ImageDataGenerator(
    rescale = 1/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
    )
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
random_datagen.fit(X_train)
# fits the model on batches with real-time data augmentation:
model.fit(random_datagen.flow(X_train, y_train, batch_size=32,
         subset='training'),
         validation_data=random_datagen.flow(X_train, y_train,
         batch_size=8, subset='validation'),
         steps_per_epoch=len(X_train) / 32, epochs=32)
         